In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
review_dict = {'name':[], 'date':[], 'rating':[], 'review':[]}
for i in range(0,39):
    url = 'https://www.metacritic.com/game/playstation-4/crystar/user-reviews?page=' + str(i)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'): 
        if review.find('div', class_='name') == None:
            break 
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'): 
            review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text)
           # print(review.find('span', class_='blurb blurb_expanded').text)
        elif review.find('div',class_='review_body').find('span') == None:
            review_dict['review'].append('No review text.')
           # print("No review")
        else:
            review_dict['review'].append(review.find('div',class_='review_body').find('span').text)

In [8]:
cs_reviews = pd.DataFrame(review_dict)
print(cs_reviews)

               name          date rating  \
0      c57v76v7v7v6  Aug 10, 2020     10   
1        Starzen517  Jul 24, 2020     10   
2   tesadesrvdbetta  Feb  5, 2020     10   
3          motymurm  Feb 19, 2020      9   
4           WAR1812  Sep  7, 2019      9   
5    Dr4gonkiller96  Oct 24, 2019      9   
6   Cageyanemone007  Sep 19, 2019     10   
7           Danisco  Sep  2, 2019     10   
8        terraxptan  Sep  1, 2019     10   
9              Rod1  Mar 28, 2020      8   
10        Valtiel86  Aug 31, 2019      7   
11    TricksterZero  Sep  8, 2019      7   
12          Hammer7  May 30, 2020      8   
13   AncapMetaleiro  May 13, 2020      6   
14       Laithherzo  Jul 26, 2020      1   
15     shamballa_69  Jul 19, 2020      0   
16       Uzumaki212  Sep 22, 2019      5   
17        DenerMelo  Jul 19, 2020      0   
18          Skapple  Aug  4, 2020      0   
19    Hugegapjungle  Jul  4, 2020      1   
20           Martma  Jul 19, 2020      1   
21  beautifulforest  Oct 14, 202